# Table detector

## Colab requirements

Before restarting runtime (remeber to select GPU runtime)$\dots$

In [ ]:
!git clone https://github.com/Wadaboa/table-detector.git
!pip install -r table-detector/init/colab_requirements.txt

After restarting runtime$\dots$

In [ ]:
import os, sys

sys.path.insert(0, "/content/table-detector")
sys.path.insert(0, "/content/table-detector/src")
os.chdir("/content/table-detector")

## Imports

In [ ]:
import torch
import wandb
import yaml
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed

import learning
import transforms
import detectors
import utils
import train
from datasets import DATASETS
from train import NUM_CLASSES

## Initialization

### Matplotlib

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 12]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

### Weights & biases

In [ ]:
!wandb login

In [ ]:
!wandb enabled

### PyTorch and numpy

In [ ]:
DEVICE = utils.get_device()
DEVICE

In [ ]:
if DEVICE.type != "cpu":
    !nvidia-smi

## Data loading

In [ ]:
with open('parameters.yml', 'r') as conf:
    args = yaml.load(conf, Loader=yaml.FullLoader)
params = utils.Struct(**args)
params.generic.device = DEVICE

In [ ]:
train_dataset, test_dataset = train.get_dataset(params)

In [ ]:
def show(img):
    np_img = utils.to_numpy(img)
    plt.imshow(np_img)

def draw_predictions(image, boxes, scores, conf_thresh=0.5, width=4):
    images = []
    for image, _ in test_dataset[index]:
        img = utils.denormalize_image(image)
        colors = generate_colors(len(boxes))
        img = utils.draw_bounding_boxes(img, boxes[scores >= conf_thresh], width=width, colors=colors)
        images.append(img)
    grid = torchvision.utils.make_grid(images)
    show(grid)

def draw_targets(index, width=4):
    images = []
    for image, target in train_dataset[index]:
        img = utils.denormalize_image(image)
        colors = generate_colors(len(target["boxes"]))
        img = utils.draw_bounding_boxes(img, target["boxes"], width=width, colors=colors)
        images.append(img)
    grid = torchvision.utils.make_grid(images)
    show(grid)

In [ ]:
interact(draw_boxes, index=widgets.IntSlider(min=0, max=len(train_dataset), step=1, value=10));

## Training

In [ ]:
!python3 src/train.py

### Testing

In [ ]:
detector = detectors.get_detector(params, NUM_CLASSES)
detector.to(params.generic.device)

In [ ]:
image, target = test_dataset[10][0]
detector.eval()
res = detector([image.to('cuda:0')])

In [ ]:
interact(
    draw_boxes, image=fixed(image), boxes=fixed(res[0]["boxes"]), scores=fixed(res[0]["scores"]), 
    conf_thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5)
);